### Hotel in Jeju Data
- '부킹닷컴'의 제주 내 호텔 성급, 가격, 위치, 후기를 스크래핑합니다.

In [171]:
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import re
import json

In [124]:
url = 'https://www.booking.com/searchresults.ko.html?aid=376440&label=bdot-SIcScZhJX6z_*YtUYg62hwS267777897793%3Apl%3Ata%3Ap1%3Ap22%2C563%2C000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-325272469656%3Alp1009871%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YUlRwjG4dAJkHxCuUKVzpFo&sid=857a41f185409d9f6d0c6b4d32c4534e&sb=1&src=searchresults&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.ko.html%3Faid%3D376440%3Blabel%3Dbdot-SIcScZhJX6z_%252AYtUYg62hwS267777897793%253Apl%253Ata%253Ap1%253Ap22%252C563%252C000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-325272469656%253Alp1009871%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9YUlRwjG4dAJkHxCuUKVzpFo%3Bsid%3D857a41f185409d9f6d0c6b4d32c4534e%3Btmpl%3Dsearchresults%3Bcity%3D-714191%3Bclass_interval%3D1%3Bdest_id%3D-714191%3Bdest_type%3Dcity%3Bdr_ps%3DIDR%3Bdtdisc%3D0%3Bfrom_idr%3D1%3Bilp%3D1%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Boffset%3D0%3Bpostcard%3D0%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrpvid%3De02638be83480079%3Bss_all%3D0%3Bssb%3Dempty%3Bsshis%3D0%3Btop_ufis%3D1%26%3B&ss=%EC%A0%9C%EC%A3%BC&is_ski_area=0&ssne=%EC%A0%9C%EC%A3%BC&ssne_untouched=%EC%A0%9C%EC%A3%BC&city=-714191&checkin_year=2021&checkin_month=4&checkin_monthday=24&checkout_year=2021&checkout_month=4&checkout_monthday=25&group_adults=2&group_children=0&no_rooms=1&from_sf=1'
path = '/Users/seyoung/2021/dss17/eda_etc/chromedriver'
driver = webdriver.Chrome(path)
driver.get(url)

source = driver.page_source
soup = BeautifulSoup(source, 'html.parser')

In [125]:
page_bar = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__pages > ul')[0]
page_bar

<selenium.webdriver.remote.webelement.WebElement (session="45ab432982fd69be27e933cd04481b42", element="8ee4f3aa-e0fe-4740-a945-a0f8f6102d21")>

In [126]:
# 전체 페이지 개수 확인
pages = page_bar.find_elements_by_css_selector('a > div.bui-u-inline')
last_pages = ''

for idx, page in enumerate(pages):
    print(page.text)
    if idx == len(pages)-1: # 마지막 페이지
        last_pages = page.text


1
2
3
4
5
9


In [127]:
# 현재 페이지 번호
def get_curr():
    curr_page = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__pages > ul > li.bui-pagination__item.bui-pagination__item--active.sr_pagination_item.current > a > div.bui-u-inline')[0]
    
    return int(curr_page.text)

In [128]:
# 성급 (성급이 표시되지 않은 호텔 제외)
def get_stars(listed):
    stars = []

    for i in range(1, len(listed)):
        try:
            like = driver.find_element_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[1]/div[1]/div[1]/span/span[1]/span/span')
            a = like.get_attribute('aria-label')
            stars.append(a[0])
            
        # 성급이 표시되지 않는 호텔 
        except Exception as NoSuchElementException:
            stars.append('')
    
    return stars

In [129]:
# 가격
def get_prices(listed):
    prices = []
    
    for i in range(1, len(listed)):
        try:
            price = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[2]/div/div/div/div/div[2]/div[1]/div[2]/div/div')[0]
            base_price = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[2]/div/div/div/div/div[2]/div[1]/div[2]/div/span')[0]
            
            # strong label로 표시된 가격 포함
            if base_price.text == '':
                base_price = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div[' + str(i) + ']/div[2]/div[2]/div/table/tbody/tr/td[2]/div[2]/strong/label')[0]
            
            prices.append(base_price.text)
            
        # 중간 베너 제외
        except Exception as IndexError:
            prices.append('')
            
    return prices

In [130]:
# 호텔명
def get_names(listed):
    names = []
    
    for i in range(1, len(listed)):
        try:
            name = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[1]/div[1]/div[1]/h3/a/span[1]')[0]
            names.append(name.text)
            
        # 중간 베너 제외
        except Exception as IndexError:
            names.append('')
            
    return names

In [131]:
def data_scraping():
    res_stars = []
    res_names = []
    res_prices = []
    
    root_div = driver.find_elements_by_css_selector('#hotellist_inner')[0]
    # 한 페이지당 호출되는 목록 수
    listed = root_div.find_elements_by_xpath('//*[@id="hotellist_inner"]/div')
    
    res_stars = get_stars(listed)
    res_names = get_names(listed)
    res_prices = get_prices(listed)
              
    return list(zip(res_names, res_stars, res_prices))

data_scraping()

[('메종 글래드 제주', '5', '기존 요금 ₩205,124 현재 요금 ₩133,250'),
 ('그랜드 하얏트 제주', '5', ''),
 ('라마다 프라자 호텔', '5', '기존 요금 ₩205,787 현재 요금 ₩175,000'),
 ('롯데시티호텔 제주', '4', '요금 ₩167,200'),
 ('라마다 제주시티호텔', '4', '기존 요금 ₩94,956 현재 요금 ₩80,750'),
 ('호텔 리젠트 마린 더블루', '4', '기존 요금 ₩217,500 현재 요금 ₩87,000'),
 ('신라스테이 제주', '4', '기존 요금 ₩242,637 현재 요금 ₩145,200'),
 ('호텔 휘슬락', '4', '기존 요금 ₩300,018 현재 요금 ₩75,000'),
 ('오션스위츠 제주 호텔', '4', '요금 ₩85,000'),
 ('베스트웨스턴 제주호텔', '4', '기존 요금 ₩150,000 현재 요금 ₩75,000'),
 ('', '', ''),
 ('호텔난타 제주', '4', '기존 요금 ₩291,000 현재 요금 ₩87,300'),
 ('캠퍼트리 호텔 & 리조트', '', '기존 요금 ₩260,158 현재 요금 ₩169,000'),
 ('제주 아길라 호텔 오시아노 스위츠', '3', '요금 ₩163,350'),
 ('호텔 시리우스', '4', '요금 ₩88,000'),
 ('제주 칼호텔', '5', '요금 ₩93,500'),
 ('제주선샤인호텔', '3', '요금 ₩57,240'),
 ('제주 센트럴 시티호텔', '3', '₩59,248'),
 ('아스타호텔', '4', '기존 요금 ₩159,000 현재 요금 ₩79,500'),
 ('유탑유블레스호텔 제주', '4', '기존 요금 ₩260,006 현재 요금 ₩111,800'),
 ('제주 마레보 비치 호텔', '3', '요금 ₩139,200'),
 ('호텔 위드 제주', '4', '요금 ₩67,000'),
 ('제주오리엔탈호텔 & 카지노', '5', '요금 ₩74,000'),
 ('호텔 

In [132]:
def get_hotelDatas():
    raw_datas = []
    
    # 스크래핑 페이지 범위 (현재 페이지 ~ 마지막 페이지)
    while get_curr() <= int(last_pages):
        # scraping
        result = data_scraping()
        raw_datas.append(result)
        # print(result)

        if get_curr() == int(last_pages):
            break

        curr_page = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__pages > ul > li.bui-pagination__item.bui-pagination__item--active.sr_pagination_item.current > a > div.bui-u-inline')[0]
        print(curr_page.text,"페이지 완료")

        move_next = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__item.bui-pagination__next-arrow > a')[0]
        move_next.send_keys('\n')

        driver.implicitly_wait(1)
        time.sleep(1+random.uniform(0,1))
        
    return raw_datas

In [224]:
# 기존 요금, 현재 요금 데이터 파싱
def price_parsing(datas):
    target_prices = []
    
    for i in range(len(datas[0])):
        target_prices.append(datas[0][i][2])

    prices = [] 

    for i in range(len(target_prices)):
        re_prices = re.findall('\d+', before_p[i])
        base_curr_prices = [] 

        if len(re_prices) == 0:
            base_curr_prices.append('')
            base_curr_prices.append('')
            
        elif len(re_prices) == 2:
            base_curr_prices.append(re_prices[0] + re_prices[1])
            base_curr_prices.append('')
            
        else:
            base_curr_prices.append(re_prices[0] + re_prices[1])
            base_curr_prices.append(re_prices[2] + re_prices[3])

        prices.append(base_curr_prices)

#     print(prices)
    return prices

price_parsing(datas)

[['79829', '52800'],
 ['48000', ''],
 ['49400', ''],
 ['49500', ''],
 ['83000', ''],
 ['54000', ''],
 ['76977', ''],
 ['77317', ''],
 ['35000', ''],
 ['', ''],
 ['', ''],
 ['', ''],
 ['31700', ''],
 ['150091', '78000'],
 ['56627', ''],
 ['43800', ''],
 ['50000', ''],
 ['44000', ''],
 ['50000', '']]

In [225]:
datas = get_hotelDatas()
re_prices = price_parsing(datas)

raw_dicts = {'hotel_name':'', 'star':'', 'base_price':'', 'cur_price':''}
raw_datas = []
hotelName_idx, star_idx, basePrice_idx, curPrice_idx = 0, 1, 2, 3

for i in range(len(datas[0])):
    
    raw_dicts['hotel_name'] = datas[0][i][hotelName_idx]
    raw_dicts['star'] = datas[0][i][star_idx]
    raw_dicts['base_price'] = re_prices[i][0]
    raw_dicts['cur_price'] = re_prices[i][1]
    
    raw_datas.append(raw_dicts.copy())
    
raw_datas

[{'hotel_name': '계룡길벽돌가',
  'star': '4',
  'base_price': '79829',
  'cur_price': '52800'},
 {'hotel_name': 'J-ONE Hotel 제이원호텔',
  'star': '2',
  'base_price': '48000',
  'cur_price': ''},
 {'hotel_name': '제주 오름펜션',
  'star': '3',
  'base_price': '49400',
  'cur_price': ''},
 {'hotel_name': '백록담펜션', 'star': '3', 'base_price': '49500', 'cur_price': ''},
 {'hotel_name': '제주 나인 리조트',
  'star': '2',
  'base_price': '83000',
  'cur_price': ''},
 {'hotel_name': '거문오름 호스텔',
  'star': '',
  'base_price': '54000',
  'cur_price': ''},
 {'hotel_name': '썬랜드 호텔', 'star': '3', 'base_price': '76977', 'cur_price': ''},
 {'hotel_name': '아뜨네 펜션', 'star': '3', 'base_price': '77317', 'cur_price': ''},
 {'hotel_name': '제주 바다성 펜션',
  'star': '3',
  'base_price': '35000',
  'cur_price': ''},
 {'hotel_name': '퓨어스테이', 'star': '', 'base_price': '', 'cur_price': ''},
 {'hotel_name': '', 'star': '', 'base_price': '', 'cur_price': ''},
 {'hotel_name': '벤티모 호텔 앤 레지던스 제주',
  'star': '4',
  'base_price': '',
  'cur_pr

---

In [1]:
# 한글 설정
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

f_path = "/Users/seyoung/Library/Fonts/AppleGothic.ttf"
font_name = font_manager.FontProperties(fname=f_path).get_name()
rc('font', family=font_name)